In [1]:
ModelsDir <- '/home/kate/Research/Property/Models/'
DataDir <- '/home/kate/Research/Property/Data/'
ModelName <- "wc_nbglm_ofst_ecy"
UseSavedIfExists <- FALSE

In [2]:
library(MASS)

In [3]:
source('/home/kate/code/Utils/MyFunctions.R')

In [4]:
training_dataset <- read.csv(paste(DataDir,"property_wcf_training.csv", sep = ""), header=TRUE)
testing_dataset <- read.csv(paste(DataDir,"property_wcf_testing.csv", sep = ""), header=TRUE)

In [5]:
formula <- cova_ic_nc_water ~ log_sqft + usagetype + log_property_age + log_water_risk_3_blk + cova_deductible + customer_cnt_active_policies 

In [6]:
kfold <- 5

In [7]:
models_attr <- data.frame()
models_coef <- data.frame()
training_dataset$nbglm_ofst_ecy_cv <- 0
testing_dataset$nbglm_ofst_ecy_cv <- 0

for (fold in 1:kfold-1){
  training_dataset[paste("nbglm_ofst_ecy_",fold,sep="")] <- 0 
  testing_dataset[paste("nbglm_ofst_ecy_",fold,sep="")] <- 0      
    }
# 
#CV-Folds
AIC_l <- list()
BIC_l <- list()
logLik_l <- list()
NWG_Test <- list()
NWG_Train <- list()
coef_l <- list()
folds_l <- list() 
i <- 1
for (fold in 1:kfold-1){
    print(paste("Fold ",fold," of ",kfold," currently processing...",sep=""))
    training_dataset_fold <- training_dataset[training_dataset[paste("fold",fold,sep="_")] > 0,]
    validation_dataset <- training_dataset[training_dataset[paste("fold",fold,sep="_")] == 0,]
    #Modeling
    ModelFile <- paste(ModelsDir,ModelName,"_",fold,".rds",sep="")
    if(file.exists(ModelFile) && UseSavedIfExists){
        nbglm <- readRDS(ModelFile)
    } else {
        nbglm <- glm.nb(formula,  data = training_dataset_fold,  offset=log_ecy,control = glm.control(maxit=10000))
        saveRDS(nbglm, ModelFile)             
    }#if modeling 
    #Training Prediction
    pred_train <- predict(nbglm,training_dataset, type="response")
    train_fold_NWG <- NormalizedWeightedGini(training_dataset$cova_ic_nc_water,pred_train,training_dataset$ecy)
    training_dataset[paste("nbglm_ofst_ecy_",fold,sep="")] <- pred_train
    training_dataset$nbglm_ofst_ecy_cv <- training_dataset$nbglm_ofst_ecy_cv + pred_train/kfold
    #Testing Prediction
    pred_test <- predict(nbglm,testing_dataset, type="response")
    test_fold_NWG <- NormalizedWeightedGini(testing_dataset$cova_ic_nc_water,pred_test,testing_dataset$ecy)
    testing_dataset$nbglm_ofst_ecy_cv <- testing_dataset$nbglm_ofst_ecy_cv + pred_test/kfold 
    testing_dataset[paste("nbglm_ofst_ecy_",fold,sep="")] <- pred_test

    #Model attributes
    folds_l[[i]] <- i
    AIC_l[[i]] <- AIC(nbglm)
    BIC_l[[i]] <- BIC(nbglm)  
    logLik_l[[i]] <- logLik(nbglm) 
    NWG_Train[[i]] <- train_fold_NWG    
    NWG_Test[[i]] <- test_fold_NWG    
    #Model coeficients
    model_coef_fold <- data.frame(
        fold = fold+1,
        pred_name = summary(nbglm)$coefficients[,0],
        estimate =  summary(nbglm)$coefficients[,1],
        StdError =  summary(nbglm)$coefficients[,2],  
        zValue =   summary(nbglm)$coefficients[,3],     
        valuePr =   summary(nbglm)$coefficients[,4]    
    )  
    #pred_name is an index, row name, convert it into column
    model_coef_fold <- cbind(name = rownames(model_coef_fold), model_coef_fold)
    rownames(model_coef_fold) <- 1:nrow(model_coef_fold) 
    #main table
    models_coef <- rbind(models_coef,model_coef_fold)         
    i <- i + 1
    }#folds loop   
    #Model attributes data frame
    models_fold_attr <- data.frame( 
        fold = unlist(folds_l),
        AIC = unlist(AIC_l),
        BIC = unlist(BIC_l),   
        logLik = unlist(logLik_l),
        TrainNormalizedWeightedGini = unlist(NWG_Train),     
        TestNormalizedWeightedGini = unlist(NWG_Test)
    )
    models_attr <- rbind(models_attr,models_fold_attr)                 

[1] "Fold 0 of 5 currently processing..."
[1] "Fold 1 of 5 currently processing..."
[1] "Fold 2 of 5 currently processing..."
[1] "Fold 3 of 5 currently processing..."
[1] "Fold 4 of 5 currently processing..."


In [8]:
head(models_attr)

,fold,AIC,BIC,logLik,TrainNormalizedWeightedGini,TestNormalizedWeightedGini
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,92059.70,92216.64,-46016.85,0.3786850,0.4081360
2,2,91948.13,92105.07,-45961.07,0.3786824,0.4089000
3,3,92038.12,92195.05,-46006.06,0.3787529,0.4088372
4,4,92009.78,92166.71,-45991.89,0.3788325,0.4091049
5,5,92168.13,92325.06,-46071.06,0.3782524,0.4082698


In [9]:
write.table(models_attr,paste(ModelsDir,ModelName,"_attr.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)

In [10]:
head(models_coef)

,name,fold,estimate,StdError,zValue,valuePr
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,(Intercept),1,-15.5208505,1.0400206,-14.923600,2.314439e-50
2,log_sqft,1,0.7028659,0.0318033,22.100408,3.132276e-108
3,usagetypePRIMARY,1,3.4023745,1.0003671,3.401126,6.710886e-04
4,usagetypeRENTAL,1,3.0982153,1.0005322,3.096567,1.957754e-03
5,usagetypeSEASONAL,1,3.0570082,1.0190544,2.999848,2.701145e-03
6,usagetypeSECONDARY,1,1.9171969,1.1556225,1.659017,9.711244e-02


In [11]:
write.table(models_coef,paste(ModelsDir,ModelName,"_coef.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)

Normalized Weighted gini Training

In [12]:
NormalizedWeightedGini(training_dataset$cova_ic_nc_water,training_dataset$nbglm_ofst_ecy_cv,training_dataset$ecy)

[1] 0.3787169

Normalized Weighted gini Testing

In [13]:
NormalizedWeightedGini(testing_dataset$cova_ic_nc_water,testing_dataset$nbglm_ofst_ecy_cv,testing_dataset$ecy)

[1] 0.4087454

In [ ]:
write.table(training_dataset,paste(DataDir,"property_wcf_training.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)
write.table(testing_dataset,paste(DataDir,"property_wcf_testing.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)
write.table(prediction_dataset,paste(DataDir,"property_water_claims_non_cat_fs.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)